In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

In [ ]:
#task 6

def BSM_exact(S0, sigma, T, r, K):
    d_plus = norm.cdf((np.log(S0 / K) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))) 
    d_minus = norm.cdf((np.log(S0 / K) + (r - (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T)))
    return S0 * d_plus - np.exp(-r * T) * K * d_minus

def BSM(S, r, sigma, dt, z):
    return r * S * dt + sigma * S * z * np.sqrt(dt)

def BSM_2(S, r, sigma, dt, z):
    return S * np.exp((r - (sigma ** 2) / 2) * dt + sigma * np.sqrt(dt) * z)

def BSM_payoff(S, K):
    if S > K:
        return S - K
    return 0

S0, r, sigma, T = 100, 0.1, 0.1, 1
K, N = 100, 10000
z = np.random.normal(0, 1, N)

S = np.zeros(N)
for i in range(N):
  S[i] = BSM_payoff(BSM_2(S0, r, sigma, T, z[i]), K) * np.exp(-r*T) 

print('BSM exact:', BSM_exact(S0, sigma, T, r, K))
print('BSM:', np.mean(S))

BSM exact: 10.308150925634422
BSM: 10.288623714285649


In [ ]:
# task 7

S0, r, sigma, T = 100, 0.1, 0.1, 1
K, period_count = 100, 1

N = 10000

S = np.zeros((N, period_count+1))
S[:, 0] = S0


z = np.zeros((N, period_count))
for c in range(N):
  z[c] = np.random.normal(0, 1, period_count)

for n in range(N):
  for i in range(period_count):
    S[n, i+1] = BSM_2(S[n, i], r, sigma, T / period_count, z[n, i])

print('asian call-option payoff: ', 
      np.mean(
          [BSM_payoff(s, K) * np.exp(-r * T) 
          for s in np.mean(S[:, 1:], axis=1)])
      )

[[100.         114.51317711]
 [100.         107.23529499]
 [100.         121.26538723]
 ...
 [100.          89.02314977]
 [100.         137.64575342]
 [100.          97.59037215]]
asian call-option payoff:  10.30578180000073


In [ ]:
# task 8

S0, r, sigma, T = 100, 0.1, 0.1, 1
K, N = 100, 10
count = 10000

S = np.zeros((count, N))
S[:, 0] = S0

def sigma_f(S, sigma):
  return sigma * (1 + 1/S)

for c in range(count):
  z = np.random.normal(0, 1, N)
  for n in range(1, N):
    S[c, n] = S[c, n - 1] + BSM(S[c, n - 1], r, sigma_f(S[c, n - 1], sigma), T / N, z[n])

print('europian call-option payoff:', np.mean(np.array([BSM_payoff(s, K) * np.exp(-r * T) for s in S[:, N-1]])))

europian call-option payoff: 9.212794403521018


In [ ]:
V_true = 1

In [ ]:
# task 5

from scipy.special import gamma

n_dim = 20 
V = (np.power(np.pi, n_dim / 2) / gamma(n_dim / 2 + 1)) / (2 ** n_dim) 
V_simplex = np.power(np.sqrt(n_dim), n_dim) / np.math.factorial(n_dim)

print(V, V_simplex)

def get_points(n, n_dim):
    points_under_line = []
    while len(points_under_line) < n:
        point = np.random.uniform(0, np.sqrt(n_dim + 1), n_dim + 1)
        point /= np.sum(point)
        point *= np.sqrt(n_dim)
        points_under_line.append(point[:-1].copy())
    return points_under_line

points_under_line = get_points(100, n_dim)
err = 10
V_apprx = 0
count_in_sphere = 0
count_all = len(points_under_line)
print("error        in sphere        all")

while err > 1e-9:
    for point in points_under_line:
        if np.sum(np.power(point, 2)) < 1:
            count_in_sphere += 1
    V_apprx = (count_in_sphere / count_all) * V_simplex
    err = np.abs(V - V_apprx) / V_true
    if err < 1e-10:
        break
    points_under_line = get_points(count_all, n_dim)
    count_all += count_all
    print(f"{err}   {count_in_sphere}   {count_all}")

In [ ]:
def tri_diag_mat_solve(mat_A, mat_b):
    n = len(mat_b)
    x = np.zeros(n)
    coef_A = np.zeros(n)
    coef_B = np.zeros(n)
    coef_A[1] = - mat_A[0][1] / mat_A[0][0]
    coef_B[1] = mat_b[0] / mat_A[0][0]
    
    for i in range(1, n - 1):
        b_gamma = mat_b[i] / mat_A[i][i - 1]
        alpha_gamma = mat_A[i][i] / mat_A[i][i - 1]
        coef_A[i + 1] =  - mat_A[i][i + 1] / ( mat_A[i][i - 1] * coef_A[i] + mat_A[i][i] )
        coef_B[i + 1] = ( b_gamma - coef_B[i] ) / ( coef_A [i] + alpha_gamma) 

    i = n - 1
    b_gamma = mat_b[i] / mat_A[i][i - 1]
    alpha_gamma = mat_A[i][i] / mat_A[i][i - 1]
    x[i] = - ( coef_B[i]  - b_gamma ) / ( coef_A[i] + alpha_gamma )
    
    for j in range(n - 1, 0 , -1):
        x[j - 1] = x[j] * coef_A[j] + coef_B[j]
    
    return x

def all_values(x_data, y_data, val):
    li = np.where(x_data < val)[0][-1]
    ri = np.where(x_data >= val)[0][0]
    if ri == val:
        return y_data[ri]
    else:
        return (y_data[ri] - y_data[li]) / (x_data[ri] - x_data[li]) * (val - x_data[li]) + y_data[li]

In [ ]:
# task 10

from itertools import combinations

def comb(n, k):
  return len(list(combinations(np.arange(n), k)))


def faure(k, dim):

  k += dim - 1
  base = dim
  if base % 2 == 0 and base != 2 and base != 1: base += 1
  print(f'base = {base}')
  while not is_prime(base):base += 2

  x = np.zeros((k, dim))
  for num in range(k): 
    for d in range(dim):
      basis = base_num(base, num)
      lb = len(basis)
      y = np.zeros(lb)
      for j in range(1,lb):
        y[j] = ( sum([comb(j - 1, l) * ((d - 1) ** (l - j + 1)) *
                  int(basis[l]) for l in range(lb)]) % base )
      x[num, d] = sum([y[j] / (base ** (j)) for j in range(1, lb)])
  return np.array(x[dim-1:])

  
def sobol(k, Ms):
  base = 2
  all_max = len(base_num(base, k))
  x = np.zeros(all_max, dtype=int)
  y = np.zeros((k, all_max), dtype=int)
  z = np.zeros(k, dtype=float)
  b = len(base_num(base, k))
  x[0], x[1], x[2] = Ms[0], Ms[1], Ms[2]
  max_len = 1
    
  for num in range(3, all_max):
    basis_1 = base_num(base, 2 * x[num-1])
    basis_2 = base_num(base, 8 * x[num-3])
    basis_3 = base_num(base, x[num-3])
    max_len = max(len(basis_1),len(basis_2),len(basis_3))
    
    basis_1 = add_zeros(basis_1, max_len)
    basis_2 = add_zeros(basis_2, max_len)
    basis_3 = add_zeros(basis_3, max_len)
    
    basis = add_bitwise([basis_1,basis_2,basis_3])
    new_num = return_num(basis, base)
    x[num] = new_num
  
  v_matr = np.zeros((all_max, all_max))
      
  for n, num in enumerate(x):
    v = base_num(base, num)
    for i, bit in enumerate(v):
      v_matr[n-i, n] += int(bit)

  
  for num in range(0, k):
    v = add_zeros(base_num(base, num+1), all_max)
    v = np.array([int(bit) for bit in v])
    
    vc = np.asarray(v, dtype = int)
    print(f'Num = {num+1}, v = {v}')
    y[num] = np.dot(v_matr, v) % 2
    z[num] = return_num(''.join(list([str(elem) for elem in y[num]])), 2, True)
      
  return z